# Prototype

In [1]:
# Imports
import pandas as pd
from binance.client import Client
from binance.enums import *
import datetime
import json

import schedule
import time
from datetime import datetime, timedelta
import features
from joblib import dump, load

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.display import clear_output

def get_api_keys(site: str, api_type: str)->str:
    """
    gets api keys stored in api-keys/api-keys.txt
    site: 'binance'
    api_type: 'api', 'secret'
    """
    with open('../api-keys/api-keys.txt') as json_file:
        return json.load(json_file)[site][api_type]
# Constants
api_key = get_api_keys("binance", "api")
api_secret = get_api_keys("binance", "secret")

client = Client(api_key=api_key, api_secret=api_secret)
sleep = 1

In [2]:
model = load('../models/model_minute_tpsl_1.0.joblib') 

In [3]:
btc = client.get_asset_balance(asset='BTC')
btc["free"]

'0.00000000'

In [4]:
def printer():
    """main printer. Fetches latest minutely data, adds features, predicts, buys/sells if necessary"""
    global sleep
    sleep = 1
    tp = 3.
    sl = 2.
    busdd = client.get_asset_balance(asset='BUSD')
    busd = float(busdd["free"]) + float(busdd["locked"])
    btcd = client.get_asset_balance(asset='BTC')
    btc_free = float(btcd["free"])
    btc_locked = float(btcd["locked"])
    btc_price = float(client.get_recent_trades(symbol='BTCBUSD', limit=1)[0]["price"])
    clear_output()
    if btc_free<(5/btc_price) and btc_locked>(5/btc_price): #Can just do == 0.
        #BTC already properly stoplossed
        send_message("waiting")
    elif btc_free>(5/btc_price): #BTC Needing to be stoplossed with oco
        order = set_oco(tp, sl, btc_price)
    else: #If not holding any BTC, see if it's time to buy
        # Download the minutely data (minimum 5000 minutes or so)
        cancel_all_orders()
        df = get_minutely_data(symbol="BTCUSDT", 
                       kline_interval=Client.KLINE_INTERVAL_1MINUTE, 
                       days=3.5)
        # Add moving average
        df["sma"] = features.get_moving_average(df.close, 5000)
        
        if btc_price > df.sma.iloc[-1]: # If price is above moving average
            df = features.add_all_features(df) 
            df.dropna(inplace=True)
            indicators = list(df.columns)[13:]
            df = df[list(indicators)].copy().astype(np.float32)
            result = model.predict(df)[-1]
            if result == 1:
                order = buy(busd)
            else:
                send_message("ml_no", btc_price)
        else: #BTC is below moving average
            send_message("belowma", btc_price, df.sma.iloc[-1])
            sleep = int(df.sma.iloc[-1]-btc_price)
    reset_client()

In [30]:
def reset_client():
    """resets the client to prevent 'read operation timed out'"""
    global client
    global api_key
    global api_secret
    client = Client(api_key=api_key, api_secret=api_secret)

def set_oco(tp, sl, price):
    """sets oco if doesn't already exist"""
    btc = client.get_asset_balance(asset='BTC')["free"]
    order = client.create_oco_order(
        symbol='BTCBUSD',
        side=SIDE_SELL,
        stopLimitTimeInForce=TIME_IN_FORCE_GTC,
        quantity=btc,
        stopPrice=str(round(price*(1-sl/100), 2)+10), 
        stopLimitPrice=str(round(price*(1-sl/100), 2)), #What if +1000 and much lower
        price=str(round(price*(1+tp/100), 2)))
    send_message("oco", str(round(price*(1+tp/100), 2)), str(round(price*(1-sl/100), 2)))
    return order
    
def buy(busd):
    """market limit order to buy with 50% of busd"""
    btc_price = float(client.get_recent_trades(symbol='BTCBUSD', limit=1)[0]["price"])
    order = client.order_limit_buy(
        symbol='BTCBUSD',
        quantity=round(busd*0.5/btc_price, 5),
        price=str(round(btc_price-10, 2)))
    send_message("bought", str(round(btc_price-10, 2)))
    return order
    
def cancel_all_orders():
    """cancels all orders--cancels previous limit buy order if it exists"""
    orders = client.get_open_orders(symbol='BTCBUSD')
    for order in orders:
        result = client.cancel_order(
            symbol='BTCBUSD',
            orderId=order["orderId"])
        
def get_minutely_data(symbol:str, kline_interval:object, start="1 Jan 1900", days=3.5):
    """
    downloads binance data
    symbol: BTCBUSD
    kline_interval: Client.KLINE_INTERVAL_1DAY, Client.KLINE_INTERVAL_1DAY, Client.KLINE_INTERVAL_1DAY
    interval_name: only used for csv name: BTCUSDT-interval_name.csv
    start: empty if from the very beginning
    """
    d = datetime.today() - timedelta(days=days)
    start_date = d.strftime("%d %b %Y %H:%M:%S")
    today = datetime.today().strftime("%d %b %Y %H:%M:%S")
    klines = client.get_historical_klines(symbol, kline_interval, start_date, today, 1000)
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE)
    data_latest = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    
    index = data.index[(data['open'] == data_latest.iloc[0].open) & (data['high'] == data_latest.iloc[0].high)].tolist()[0]
    result = pd.concat([data[:index], data_latest], ignore_index=True, sort=False)
    return result

In [31]:
# df = get_minutely_data(symbol="BTCUSDT", 
#                        kline_interval=Client.KLINE_INTERVAL_1MINUTE, 
#                        days=10)
# df = features.add_all_features(df) 
# df.dropna(inplace=True)
# indicators = list(df.columns)[12:]
# indicators.append('close')
# df = df[list(indicators)].copy().astype(np.float32)

# df.to_csv('../data/debugging.csv', index=False)

In [32]:
# df

In [33]:
schedule.clear()
schedule.every().minute.at(":01").do(printer)

Every 1 minute at 00:00:01 do printer() (last run: [never], next run: 2021-06-13 16:06:01)

In [36]:
# Discord # Saving message
def send_message(m:str, a="", b=""):
    data = {"message":"ERROR"}
    if m == "waiting":
        data["message"] = "awaiting tp and sl orders to fulfill"
        print("awaiting tp and sl orders to fulfill")
    elif m == "oco":
        data["message"] = "@everyone Set take profit $" + str(a) + " and stop loss $" + str(b)
    elif m == "belowma":
        data["message"] = "Price $" + str(a) + " is below moving average $" + str(round(b, 2)) + " by $" + str(round(b-a, 2))
    elif m == "bought":
        data["message"] = "@everyone Just bought at $" + str(a)
    elif m == "ml_no":
        data["message"] = "ML model says no at current price $" + str(a)
    with open('message.txt', 'w') as outfile:
        json.dump(data, outfile)

In [37]:
sleep = 1
while True:
    clear_output()
    schedule.run_pending()
    print(f"sleeping for {sleep} seconds")
    time.sleep(sleep)

sleeping for 1 seconds


KeyboardInterrupt: 